# N2V Prediction
Please run the 'N2V-Convallaria-1-Training' notebook first.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from tifffile import imread

from pn2v import utils
from pn2v import utils
from pn2v import prediction

# See if we can use a GPU
device=utils.getDevice()

### Load Data

In [ ]:
# We need the training data in order to calulate 'mean' and 'std' for normalization
path='data/Convallaria_diaphragm/'

# Load the test data
dataTest=imread(path+"20190520_tl_25um_50msec_05pc_488_130EM_Conv.tif")[:,:512,:512]
# We are loading only a sub image to spped up computation

# We estimate the ground truth by averaging.
dataTestGT=np.mean(dataTest[:,...],axis=0)[np.newaxis,...]


### Load the Network

In [ ]:
# Load the network, created in the 'Convallaria-2-Training' notebook
net=torch.load(path+"/last_conv_N2V.net")

### Evaluation

In [ ]:
# Now we are processing data and calculating PSNR values.
n2vRes=[]
resultImgs=[]
inputImgs=[]

# We iterate over all test images.
for index in range(dataTest.shape[0]):
    
    im=dataTest[index]
    gt=dataTestGT[0] # The ground truth is the same for all images
    
    # We are using tiling to fit the image into memory
    # If you get an error try a smaller patch size (ps)
    n2vResult = prediction.tiledPredict(im, net ,ps=256, overlap=48,
                                            device=device, noiseModel=None)
    
    
    inputImgs.append(im)

    rangePSNR=np.max(gt)-np.min(gt)
    n2vPrior=utils.PSNR(gt, n2vResult,rangePSNR )
    n2vRes.append(n2vPrior)

    print ("image:",index)
    print ("PSNR input",utils.PSNR(gt, im, rangePSNR))
    print ("PSNR N2V",n2vPrior) # Without info from masked pixel
    print ('-----------------------------------')
    
    
# We display the results for the last test image       
vmi=np.percentile(gt,0.01)
vma=np.percentile(gt,99)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.title(label='Input Image')
plt.imshow(im, vmax=vma, vmin=vmi, cmap='magma')

plt.subplot(1, 2, 2)
plt.title(label='N2V result')
plt.imshow(n2vResult, vmax=vma, vmin=vmi, cmap='magma')

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.title(label='Input Image')
plt.imshow(im[100:200,150:250], vmax=vma, vmin=vmi, cmap='magma')

plt.subplot(1, 2, 2)
plt.title(label='N2V result')
plt.imshow(n2vResult[100:200,150:250], vmax=vma, vmin=vmi, cmap='magma')
plt.show()

print("Avg PSNR:", np.mean(np.array(n2vRes) ), '+-(2SEM)',2*np.std(np.array(n2vRes) )/np.sqrt(float(len(n2vRes)) ) )